# Sequencing data analysis

### IMPORTANT: Please make sure that your are using the bash kernel to run this notebook.

### This notebook covers analysis of DNA sequencing data from raw files to processed signals.

Although this analysis is for ATAC-seq data, many of the steps (especially the first section) are the same for other types of DNA sequencing experiments.

We'll be doing the analysis in Bash, which is the standard language for UNIX command-line scripting.

## Part 1: Setting up the data

We start with raw `.fastq.gz` files, which are provided by the sequencing instrument. For each DNA molecule (read) that was sequenced, they provide the nucleotide sequence, and information about the quality of the signal of that nucleotide.

In [19]:
### Set up variables storing the location of our data
##These are the same as you generated in Notebook 1.3
##If you have already logged in and out of jupyterhub, this step may be redundant, be we add it in for good measure. 

export SUNETID="$(whoami)"
export WORK_DIR="/tc2016/${SUNETID}"
export DATA_DIR="${WORK_DIR}/data"
export FASTQ_DIR="${DATA_DIR}/fastq/"
export SRC_DIR="${WORK_DIR}/src"
export ANALYSIS_DIR="${WORK_DIR}/analysis/"
export YEAST_DIR="/srv/scratch/training_camp/saccer3/seq"
export YEAST_INDEX="/srv/scratch/training_camp/saccer3/bowtie2_index/saccer3"
export YEAST_CHR="/srv/scratch/training_camp/saccer3/sacCer3.chrom.sizes"
export TMP="${WORK_DIR}/tmp"
export TEMP=$TMP 
export TMPDIR=$TMP


Now, let's check exactly which fastqs we have:

(recall that the `ls` command lists the contents of a directory)

In [2]:
ls $FASTQ_DIR

Ct-1h_S5_L001_R1_001.fastq.gz	 DMSO-3h_S13_L001_R1_001.fastq.gz
Ct-1h_S5_L001_R2_001.fastq.gz	 DMSO-3h_S13_L001_R2_001.fastq.gz
Ct-3h_S12_L001_R1_001.fastq.gz	 Kz-1h_S1_L001_R1_001.fastq.gz
Ct-3h_S12_L001_R2_001.fastq.gz	 Kz-1h_S1_L001_R2_001.fastq.gz
Cu-1h_S4_L001_R1_001.fastq.gz	 Kz-3h_S8_L001_R1_001.fastq.gz
Cu-1h_S4_L001_R2_001.fastq.gz	 Kz-3h_S8_L001_R2_001.fastq.gz
Cu-3h_S11_L001_R1_001.fastq.gz	 Mz-1h_S3_L001_R1_001.fastq.gz
Cu-3h_S11_L001_R2_001.fastq.gz	 Mz-1h_S3_L001_R2_001.fastq.gz
Cz-1h_S2_L001_R1_001.fastq.gz	 Mz-3h_S10_L001_R1_001.fastq.gz
Cz-1h_S2_L001_R2_001.fastq.gz	 Mz-3h_S10_L001_R2_001.fastq.gz
Cz-3h_S9_L001_R1_001.fastq.gz	 WT-1h_S7_L001_R1_001.fastq.gz
Cz-3h_S9_L001_R2_001.fastq.gz	 WT-1h_S7_L001_R2_001.fastq.gz
DMSO-1h_S6_L001_R1_001.fastq.gz  WT-3h_S14_L001_R1_001.fastq.gz
DMSO-1h_S6_L001_R2_001.fastq.gz  WT-3h_S14_L001_R2_001.fastq.gz


As a sanity check, we can also look at the size and last edited time of some of the fastqs by addind `-lrth` to the `ls` command:

In [3]:
ls -lrth $FASTQ_DIR | head

total 112K
lrwxrwxrwx 1 ubuntu ubuntu 80 Sep 12 02:21 WT-3h_S14_L001_R2_001.fastq.gz -> ../../../../srv/scratch/training_camp/data/fastqs/WT-3h_S14_L001_R2_001.fastq.gz
lrwxrwxrwx 1 ubuntu ubuntu 80 Sep 12 02:21 WT-3h_S14_L001_R1_001.fastq.gz -> ../../../../srv/scratch/training_camp/data/fastqs/WT-3h_S14_L001_R1_001.fastq.gz
lrwxrwxrwx 1 ubuntu ubuntu 79 Sep 12 02:21 WT-1h_S7_L001_R2_001.fastq.gz -> ../../../../srv/scratch/training_camp/data/fastqs/WT-1h_S7_L001_R2_001.fastq.gz
lrwxrwxrwx 1 ubuntu ubuntu 79 Sep 12 02:21 WT-1h_S7_L001_R1_001.fastq.gz -> ../../../../srv/scratch/training_camp/data/fastqs/WT-1h_S7_L001_R1_001.fastq.gz
lrwxrwxrwx 1 ubuntu ubuntu 80 Sep 12 02:21 Mz-3h_S10_L001_R2_001.fastq.gz -> ../../../../srv/scratch/training_camp/data/fastqs/Mz-3h_S10_L001_R2_001.fastq.gz
lrwxrwxrwx 1 ubuntu ubuntu 80 Sep 12 02:21 Mz-3h_S10_L001_R1_001.fastq.gz -> ../../../../srv/scratch/training_camp/data/fastqs/Mz-3h_S10_L001_R1_001.fastq.gz
lrwxrwxrwx 1 ubuntu ubuntu 79 Sep 12 02:21 Mz

Let's also inspect the format of one of the fastqs. Notice that each read takes up 4 lines:
1. the read name
2. the read's nucleotide sequence
3. a '+' to indicate the record contains another line
4. a quality score for each base (a number encoded as a letter)

In [4]:
zcat $(ls $FASTQ_DIR* | head -n 1) | head -n 8

@M00653:168:000000000-ATAU0:1:2109:21182:1054 1:N:0:5
CATTTAGTACACCGTTTCCTTCANTTAACCATCCTTAAATCTGTCTCTTATACACATCTCCGAGCCCACGAGACGG
+
CCCCCGGGGGGGCFGCFGGGGGG#:C@FFGGGDECGCFGF9,EFE@FGGFGGGGGGGGGGGGDEEFEGGGGGCCFE
@M00653:168:000000000-ATAU0:1:2109:17670:1054 1:N:0:5
GTATAGAGACTAGGCAGATCTGANGATCACCTAGCGACTCTCTCCACCGTTTGACGAGGCCATTTACAAAAACATA
+
CCCCCFGCCGGG@FDGCCAF<EF#:CFFGGEFGGFGGGDGFF9FGE,FGFFFGGFGGDEEFE,?C?F,AFFFFGEF

gzip: stdout: Broken pipe


## Part 2: Adapter trimming

- In many kinds of DNA and RNA sequencing experiments, sometimes the sequences will read through the targeted sequence insert and into sequencing adapter or PCR primer sequences on the end of the fragment. When the insert size is shorter than the read length (like in some of our ATAC-seq reads), the adapter sequence is read by the sequencer.

- We need to remove such adapter sequences because they won't align to the genome.

- In ATAC-seq (the data we're analyzing), the fragment length follows a periodic distribution. Some reads have very short inserts (only a few basepairs), while other reads have inserts that are much longer (100's of basepairs — much longer than the 77bp reads we're using to read them.

- We know ahead of time that the first part of the adapter sequence is `CTGTCTCTTATA`, since our reads are sequenced using a Nextera sample prep kit.

In [5]:
# Let's sanity check our adapter sequence by seeing
# how many times it occurs in the first 100000 reads.

ADAPTER="CTGTCTCTTATA"

NUM_LINES=400000  # 4 * num_reads, since each fastq entry is 4 lines

zcat $(ls $FASTQ_DIR*R1* | head -n 1) | head -n $NUM_LINES | grep $ADAPTER | wc -l

16800

gzip: stdout: Broken pipe


In [6]:
# Let's also check how often a permutation (rearrangement)
# of the adapter sequence occurs:

NOT_ADAPTER="CGTTCTTCTATA"  # A permutation of the adapter sequence

zcat $(ls $FASTQ_DIR*R1* | head -n 1) | head -n $NUM_LINES | grep $NOT_ADAPTER | wc -l

0

gzip: stdout: Broken pipe


Notice that the correct adapter sequence occurs *many* times more in the reads than a permutation of the adapter sequene — this is an important validation that we have the right sequence.

Now, we'll trim the paired-end reads using a tool called `cutadapt`:

In [7]:
for R1_fastq in ${FASTQ_DIR}*_R1*fastq.gz; do
    
    # Get the read 2 fastq file from the filename of read 1
    R2_fastq=$(echo $R1_fastq | sed -e 's/R1/R2/')
    
    # Generate names for the trimmed fastq files

    trimmed_R1_fastq=$ANALYSIS_DIR$(echo $(basename $R1_fastq)| sed -e 's/.fastq.gz/.trimmed.fastq.gz/')
    trimmed_R2_fastq=$ANALYSIS_DIR$(echo $(basename $R2_fastq)| sed -e 's/.fastq.gz/.trimmed.fastq.gz/')   
    echo cutadapt -m 5 -e 0.20 -a CTGTCTCTTATA -A CTGTCTCTTATA \
        -o ${trimmed_R1_fastq} \
        -p ${trimmed_R2_fastq} \
        $R1_fastq \
        $R2_fastq
    cutadapt -m 5 -e 0.20 -a CTGTCTCTTATA -A CTGTCTCTTATA \
        -o ${trimmed_R1_fastq} \
        -p ${trimmed_R2_fastq} \
        $R1_fastq \
        $R2_fastq

done

cutadapt -m 5 -e 0.20 -a CTGTCTCTTATA -A CTGTCTCTTATA -o /tc2016/ubuntu/analysis/Ct-1h_S5_L001_R1_001.trimmed.fastq.gz -p /tc2016/ubuntu/analysis/Ct-1h_S5_L001_R2_001.trimmed.fastq.gz /tc2016/ubuntu/data/fastq/Ct-1h_S5_L001_R1_001.fastq.gz /tc2016/ubuntu/data/fastq/Ct-1h_S5_L001_R2_001.fastq.gz
This is cutadapt 1.10 with Python 3.5.2
Command line parameters: -m 5 -e 0.20 -a CTGTCTCTTATA -A CTGTCTCTTATA -o /tc2016/ubuntu/analysis/Ct-1h_S5_L001_R1_001.trimmed.fastq.gz -p /tc2016/ubuntu/analysis/Ct-1h_S5_L001_R2_001.trimmed.fastq.gz /tc2016/ubuntu/data/fastq/Ct-1h_S5_L001_R1_001.fastq.gz /tc2016/ubuntu/data/fastq/Ct-1h_S5_L001_R2_001.fastq.gz
Trimming 2 adapters with at most 20.0% errors in paired-end mode ...
Finished in 31.47 s (205 us/read; 0.29 M reads/minute).

=== Summary ===

Total read pairs processed:            153,618
  Read 1 with adapter:                  41,521 (27.0%)
  Read 2 with adapter:                  41,872 (27.3%)
Pairs that were too short:                 142 (0.1%

## Part 3: Alignment

Now, we're ready to align our trimmed reads to the Yeast SacCer3 reference genome.

We'll use [Bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml), which is a [Burrows-Wheeler](https://en.wikipedia.org/wiki/Burrows%E2%80%93Wheeler_transform) based spliced aligner.

Bowtie2 outputs a SAM (Sequence Alignment Map) file, which is a standard text encoding. To save space, we'll use `samtools view -b` to encode the output as a binarized SAM file — a BAM file.

In [21]:
export bowtie_index=$YEAST_INDEX
echo $bowtie_index

#create a directory to store the aligned data 
export ALIGNMENT_DIR="$ANALYSIS_DIR/aligned/"
[[ ! -d $ALIGNMENT_DIR ]] && mkdir -p "$ALIGNMENT_DIR"

/srv/scratch/training_camp/saccer3/bowtie2_index/saccer3


In [22]:
for trimmed_fq1 in ${ANALYSIS_DIR}*_R1*fastq.gz; do

    trimmed_fq2=$(echo $trimmed_fq1 | sed -e 's/_R1/_R2/')
    
    bam=$(echo "${ALIGNMENT_DIR}${trimmed_fq1##*/}" | sed -e 's/.fastq.gz/.bam/')
    bowtie2 -X2000 --mm --threads 5 -x $bowtie_index -1 $trimmed_fq1 -2 $trimmed_fq2 | samtools view -bS - > $bam        
done

[samopen] SAM header is present: 17 sequences.
153476 reads; of these:
  153476 (100.00%) were paired; of these:
    7206 (4.70%) aligned concordantly 0 times
    75741 (49.35%) aligned concordantly exactly 1 time
    70529 (45.95%) aligned concordantly >1 times
    ----
    7206 pairs aligned concordantly 0 times; of these:
      1000 (13.88%) aligned discordantly 1 time
    ----
    6206 pairs aligned 0 times concordantly or discordantly; of these:
      12412 mates make up the pairs; of these:
        10043 (80.91%) aligned 0 times
        232 (1.87%) aligned exactly 1 time
        2137 (17.22%) aligned >1 times
96.73% overall alignment rate
[samopen] SAM header is present: 17 sequences.
450816 reads; of these:
  450816 (100.00%) were paired; of these:
    18997 (4.21%) aligned concordantly 0 times
    190331 (42.22%) aligned concordantly exactly 1 time
    241488 (53.57%) aligned concordantly >1 times
    ----
    18997 pairs aligned concordantly 0 times; of these:
      2522 (13.2

## Part 4: Finding duplicate reads and alignment filtering

During sequencing, we perform PCR, which can lead to duplicate reads. In many kinds of DNA sequencing, we want to remove duplicates so that we don't double-count signal originating from the same molecule.

To do so, we use an algorithm called `sambamba` that looks for reads that mapped to exactly the same places in the genome. We also need to sort the aligned files before we can mark duplicates, since we need reads aligned to the same position to be next to each other in the file.

Bowtie2 also sets certian labels (or "flags") in the resulting alignment file to indicate information like the score of the alignment, the orientation of both mates of the fragment, and other details.

We can use these flags as a way to discard low-quality reads. [This website](https://broadinstitute.github.io/picard/explain-flags.html) provides a convenient way to interpret the meaning of these bitwise flags; for conveninece they can be encoded as numbers.

Here, we want to filter reads that fall into any of the following categories:
- the read wasn't mapped to the genome
- the read's mate wasn't mapped to the genome
- the alignment reported is not the primary alignment (it is a "runner-up" alignment)
- the read was marked as "low-quality" by the sequencer software
- the read has a mapping quality less than 30

In [23]:
for bam_file in ${ALIGNMENT_DIR}*.bam; do

    nodup_bam_file=$(echo $bam_file | sed -e 's/.bam/.nodup.bam/')
    
    # Sort and remove duplicates
    sambamba sort -m 4G -t 2 -u $bam_file | \
        sambamba markdup -l 0 -t 2 /dev/stdin /dev/stdout |\
        samtools view -F 1804 -f 2 -q 30 /dev/stdin $nodup_bam_file
    
done

[bam_header_read] EOF marker is absent. The input is probably truncated.
sambamba: command not found
sambamba: command not found
[bam_header_read] invalid BAM binary header (this is not a BAM file).
[main_samview] fail to read the header from "/dev/stdin".
[bam_header_read] EOF marker is absent. The input is probably truncated.
sambamba: command not found
sambamba: command not found
[bam_header_read] invalid BAM binary header (this is not a BAM file).
[main_samview] fail to read the header from "/dev/stdin".
[bam_header_read] EOF marker is absent. The input is probably truncated.
sambamba: command not found
sambamba: command not found
[bam_header_read] invalid BAM binary header (this is not a BAM file).
[main_samview] fail to read the header from "/dev/stdin".
[bam_header_read] EOF marker is absent. The input is probably truncated.
sambamba: command not found
sambamba: command not found
[bam_header_read] invalid BAM binary header (this is not a BAM file).
[main_samview] fail to read th

## Part 5: Peak calling

Now that we've aligned our reads to the genome and filtered the alignments, we want to identify areas of locally enriched signals, or "peaks".

For ATAC-seq, peaks correspond to accessible regions. They can include promoters, enhancers, and other regulatory regions.

We'll call peaks using [MACS2](http://liulab.dfci.harvard.edu/MACS/)

In [ ]:
PEAKS_DIR="${ANALYSIS_DIR}peaks/"
[[ ! -d $PEAKS_DIR ]] && mkdir -p "$PEAKS_DIR"

SacCer3GenSz=12157105  # The sum of the sizes of the chromosomes in the SacCer3 genome

Macs2PvalThresh="0.05"  # The p-value threshold for calling peaks 

Macs2SmoothWindow=150  # The window size to smooth alignment signal over

Macs2ShiftSize=$(python -c "print(${Macs2SmoothWindow}*0.5)")

for nodup_bam_file in ${ALIGNMENT_DIR}*.nodup.bam; do
    
    # First, we need to convert each bam to a .tagAlign,
    # which just contains the start/end positions of each read:
    
    tagalign_file=$(echo $nodup_bam_file | sed -e 's/.bam/.tagAlign.gz/')
    bedtools bamtobed -i $bam | awk 'BEGIN{OFS="\t"}{$4="N";$5="1000";print $0}' | gzip -c > $tagalign_file
    
    # Now, we can run MACS:
    output_prefix=$(echo $trimmed_fq1 | sed -e 's/.bam/.tagAlign.gz/')
    macs2 callpeak \
        -t $tagalign_file -f BED -n "$prefix" -g "$SacCer3GenSz" -p $Macs2PvalThresh \
        --nomodel --shift -$Macs2ShiftSize --extsize $Macs2SmoothWindow -B --SPMR --keep-dup all --call-summits
    
done